In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
from sklearn.feature_extraction.text import CountVectorizer
import io
import time
from io import StringIO
import re
from nltk.corpus import stopwords
from PIL import *
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import files


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
upload = files.upload()

Saving conjuctivitis.csv to conjuctivitis.csv


In [ ]:
df1 = pd.read_csv('conjuctivitis.csv', encoding = "ISO-8859-1", error_bad_lines=False)
df1.head(10)

,text,category,cat2
0,RT @UnicornDoc: When you have a stye in one ey...,1,1
1,When you have a stye in one eye and conjunctiv...,1,1
2,@nimmajji Get conjunctivitis from somewhere.. ...,0,0
3,RT @master_usmle: Streptococcus pneumoniae: di...,0,0
4,Actually so grateful for these eye drops the o...,2,1
5,To top off my shit week I've woke up this morn...,1,1
6,"Now I have conjunctivitis in both eyes, so the...",1,1
7,I think Archie has conjunctivitis in his littl...,1,1
8,I don't wanna wear mascara just incase I still...,1,1
9,Guess who now has conjunctivitis. It's me. Tha...,1,1


In [ ]:
df1.drop(['category', 'cat2'],axis=1,inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 1 columns):
text    656 non-null object
dtypes: object(1)
memory usage: 5.2+ KB


In [ ]:
df1['label'] = 'conjunctivitis'
df1.head()

,text,label
0,RT @UnicornDoc: When you have a stye in one ey...,conjunctivitis
1,When you have a stye in one eye and conjunctiv...,conjunctivitis
2,@nimmajji Get conjunctivitis from somewhere.. ...,conjunctivitis
3,RT @master_usmle: Streptococcus pneumoniae: di...,conjunctivitis
4,Actually so grateful for these eye drops the o...,conjunctivitis


In [ ]:
df1 = df1.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)# replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df1['text'] = df1['text'].apply(clean_text)
df1['text'] = df1['text'].str.replace('\d+', '')

In [ ]:
df1.head(5)

,text,label
0,rt unicorndoc stye one eye conjunctivitis,conjunctivitis
1,stye one eye conjunctivitis,conjunctivitis
2,nimmajji get conjunctivitis somewhere skodithala,conjunctivitis
3,rt master_usmle streptococcus pneumoniae disea...,conjunctivitis
4,actually grateful eye drops opticians told get...,conjunctivitis


In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [ ]:
docs = df1.text.values.tolist()

In [ ]:
str1 = ''.join(docs)

In [ ]:
text = str1
tr4w = TextRank4Keyword()
tr4w.analyze(text, window_size=4, lower=False)
tr4w.get_keywords()

conjunctivitis - 73.47156107513536
tco - 37.68325520264792
https - 29.146393221128882
eye - 21.132436956696
eyes - 14.205859364641661
amp - 6.577460748319699
day - 6.382485799521736
infection - 5.7320054363170305
week - 5.137783783881029
site - 4.960238527476208
ear - 4.2424391580837035
days - 4.17064332809176


In [ ]:
!pip install yake

     |████████████████████████████████| 61kB 4.1MB/s 
     |████████████████████████████████| 81kB 5.5MB/s 
     |████████████████████████████████| 143kB 44.8MB/s 
     |████████████████████████████████| 245kB 35.7MB/s 
     |████████████████████████████████| 655kB 43.9MB/s 
  Created wheel for yake: filename=yake-0.3.7-py2.py3-none-any.whl size=44591 sha256=6c8922b424ec03c11010b0a46fe8f5308d476c199b22422759d5b1ebcf552d28
  Stored in directory: /root/.cache/pip/wheels/90/7d/56/63a3a3b064e9214e29616880dc4170a300ca68643728ce24ba
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=e2e52e6d1a8672ec438ca0358f509976da21c395871b1bd235baa8bc4f800c5e
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for segtok: filename=segtok-1.5.7-cp36-none-any.whl size=23258 sha256=919dfc002a83fc1e15f41cc01077efbe4b04b0f31fe694fa644d786ec1e5255f
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/61

In [ ]:
import yake

text = str1

# assuming default parameters
simple_kwextractor = yake.KeywordExtractor()
keywords = simple_kwextractor.extract_keywords(text)

for kw in keywords:
	print(kw)

# specifying parameters
max_ngram_size = 1
custom_kwextractor = yake.KeywordExtractor(lan="en", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1,  features=None)

keywords = custom_kwextractor.extract_keywords(text)

for kw in keywords:
	print(kw)

(3.3349257136441313e-06, 'conjunctivitis https tco')
(8.322181636648834e-06, 'step https tco')
(8.987130950349055e-06, 'https tco')
(9.97242574314435e-06, 'usmle step https')
(1.1779296076905653e-05, 'fever rash joint')
(1.2411071027765599e-05, 'site sore eyes')
(1.3194390007975259e-05, 'eyes https tco')
(1.3813518570797514e-05, 'rash joint pain')
(1.4876879389832703e-05, 'conjunctivitis red eyes')
(1.4953488547785955e-05, 'joint pain amp')
(1.5281136359580238e-05, 'tincture https tco')
(1.5918100686710396e-05, 'include fever rash')
(1.6468982760424864e-05, 'crash usmle step')
(1.750975527030002e-05, 'natural eye drop')
(1.7992203239472225e-05, 'eye drop tincture')
(1.8137581437258857e-05, 'drop tincture https')
(1.8234041848461828e-05, 'amp conjunctivitis red')
(1.8716204380254252e-05, 'suggestions pink eye')
(1.9353479963674584e-05, 'mild include fever')
(1.964246621963813e-05, 'pain amp conjunctivitis')
(9.132140546856105e-05, 'conjunctivitis')
(9.798529050235849e-05, 'https')
(0.00